In [ ]:
import os

import keras

import random

import shutil

import tensorflow as tf

from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from shutil import copyfile

from os import getcwd

from os import listdir

import cv2

from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense

from tensorflow.keras.models import Model, load_model

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

import imutils

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.image  as mpimg

In [ ]:
def data_summary(main_path):

    withmask_path = main_path+'withmaskreal'

    withoutmask_path = main_path+'withoutmaskreal'

    # number of (images) that are in the the folder named 'with_mask' 

    m_pos = len(listdir('C:/experiment/data/with_mask'))

    # number of (images) that are in the the folder named 'without_mask' 

    m_neg = len(listdir('C:/experiment/data/without_mask'))

    # number of all examples

    m = (m_pos+m_neg)

    pos_prec = (m_pos* 100.0)/ m

    neg_prec = (m_neg* 100.0)/ m

    print(f"Number of examples: {m}")

    print(f"Percentage of positive examples: {pos_prec}%, number of pos examples: {m_pos}") 

    print(f"Percentage of negative examples: {neg_prec}%, number of neg examples: {m_neg}")    
    augmented_data_path = 'C:/experiment/dest_folder/val'    

data_summary(augmented_data_path)

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

    dataset = []  

    for unitData in os.listdir(SOURCE):

        data = SOURCE + unitData

        if(os.path.getsize(data) > 0):

            dataset.append(unitData)

        else:

            print('Skipped ' + unitData)

            print('Invalid file i.e zero size')  

    train_set_length = int(len(dataset) * SPLIT_SIZE)

    test_set_length = int(len(dataset) - train_set_length)

    shuffled_set = random.sample(dataset, len(dataset))

    train_set = dataset[0:train_set_length]

    test_set = dataset[-test_set_length:]     

    for unitData in train_set:

        temp_train_set = SOURCE + unitData

        final_train_set = TRAINING + unitData

        copyfile(temp_train_set, final_train_set)  

    for unitData in test_set:

        temp_test_set = SOURCE + unitData

        final_test_set = TESTING + unitData

        copyfile(temp_test_set, final_test_set)      

WITHMASK_SOURCE_DIR = "C:/experiment/dest_folder/val/with_mask/"

TRAINING_WITHMASK_DIR = "C:/experiment/dest_folder/train/with_mask/"

TESTING_WITHMASK_DIR = "C:/experiment/dest_folder/test/without_mask/"

WITHOUTMASK_SOURCE_DIR = "C:/experiment/dest_folder/val/without_mask/"

TRAINING_WITHOUTMASK_DIR = "C:/experiment/dest_folder/train/without_mask/"

TESTING_WITHOUTMASK_DIR = "C:/experiment/dest_folder/test/without_mask/"

split_size = .8

split_data(WITHMASK_SOURCE_DIR, TRAINING_WITHMASK_DIR, 

TESTING_WITHMASK_DIR, split_size)

split_data(WITHOUTMASK_SOURCE_DIR, TRAINING_WITHOUTMASK_DIR, 

TESTING_WITHOUTMASK_DIR, split_size)

In [ ]:
print("number of images with facemask in the training set'with_mask':", len(os.listdir('C:/experiment/dest_folder/train/with_mask')))

print("number of images with facemask in the test set'with_mask':", len(os.listdir('C:/experiment/dest_folder/test/with_mask')))

print("number of images without facemask in the training set'without_mask':", len(os.listdir('experiment/dest_folder/train/without_mask')))

print("number of images without facemask in the test set'without_mask':",

len(os.listdir('experiment/dest_folder/test/without_mask')))

In [ ]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(100, (3,3), activation='relu', 

    input_shape=(150, 150, 3)),

    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),

    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(50, activation='relu'),

    tf.keras.layers.Dense(2, activation='softmax')

])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
TRAINING_DIR = "C:/experiment/dest_folder/train"

train_datagen = ImageDataGenerator(rescale=1.0/255,

                                   rotation_range=40,

                                   width_shift_range=0.2,

                                   height_shift_range=0.2,

                                   shear_range=0.2,

                                   zoom_range=0.2,

                                   horizontal_flip=True,

                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 

                                                    batch_size=10, 

                                                    target_size=(150, 150))

VALIDATION_DIR = "C:/experiment/dest_folder/test"

validation_datagen = ImageDataGenerator(rescale=1.0/255)
 

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 

                                                         batch_size=10, 

                                                         target_size=(150, 150))

checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [ ]:
istory = model.fit_generator(train_generator,

                              epochs=30,

                              validation_data=validation_generator,

                              callbacks=[checkpoint])

model.save('my_model_facemask.h5')

In [ ]:
# import the necessary libraries

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.models import load_model

import argparse

from google.colab.patches import cv2_imshow

from natsort import natsorted, ns
 

input_file_path = "C:/experiment/data/with_mask/0-with-mask.jpg"
 

print("[INFO] loading face detector model...")

prototxtPath = "C:/experiment/deploy.prototxt"

weightsPath = "C:/experiment/res10_300x300_ssd_iter_140000.caffemodel"

net = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector

print("[INFO] loading face mask detector model...")

model = load_model("C:/experiment/mymodel.h5")

def process_images(input_file_path):

  # load the input imag

  # dimensions

  image = cv2.imread(input_file_path)

  (h, w) = image.shape[:2]
 

  # construct a blob from the image

  blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),

    (104.0, 177.0, 123.0))

 # pass the blob through the network and obtain the face detections

  print("[INFO] computing face detections...")

  net.setInput(blob)

  detections = net.forward()

  # loop over the detections

  for i in range(0, detections.shape[2]):

    # extract the confidence associated with

    # the detection

    confidence = detections[0, 0, i, 2]

    # filter out weak detections by ensuring the confidence is

    # greater than the minimum confidence

    if confidence > 0.5:

      # compute the (x, y)-coordinates of the bounding box for

      # the object

      box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])

      (startX, startY, endX, endY) = box.astype("int")

      # ensure the bounding boxes fall within the dimensions of

      # the frame

      (startX, startY) = (max(0, startX), max(0, startY))

      (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

      # extract the face ROI, convert it from BGR to RGB channel

      # ordering, resize it to 150x150, and preprocess it

      face = image[startY:endY, startX:endX]

      face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

      face = cv2.resize(face, (150, 150))

      face = img_to_array(face)

      face = preprocess_input(face)

      face = np.expand_dims(face, axis=0)

      # pass the face through the model to determine if the face

      # has a mask or not

      (mask, withoutMask) = model.predict(face)[0]

      # determine the class label and color we'll use to draw

      # the bounding box and text

      label = "Mask" if mask > withoutMask else "No Mask"

      color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

   # include the probability in the label

      label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

      # display the label and bounding box rectangle on the output

      # frame

      cv2.putText(image, label, (startX, startY - 10),

        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)

      cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

  # show the output image

  save_path = input_file_path.split(".",1)

  cv2_imshow(image)

#start the process

process_images(input_file_path)